In [1]:
cd drive/MyDrive/JAIST/Research/RL/movie-lens-simulator

/content/drive/MyDrive/JAIST/Research/RL/movie-lens-simulator


In [2]:
!make style
!make setup

pip install -r ./requirements/style.txt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 103 kB 15.0 MB/s 
     |████████████████████████████████| 1.4 MB 65.9 MB/s 
     |████████████████████████████████| 61 kB 525 kB/s 
     |████████████████████████████████| 843 kB 58.9 MB/s 
     |████████████████████████████████| 96 kB 5.0 MB/s 
     |████████████████████████████████| 41 kB 446 kB/s 
     |████████████████████████████████| 66 kB 4.9 MB/s 
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.12.0
    Uninstalling importlib-metadata-4.12.0:
      Successfully uninstalled importlib-metadata-4.12.0
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This 

In [3]:
from src.data.movielens1M import LoadMovielens1M, PreprocessMovielens1M
from src.dataloader.movielens import MovieLensDataset
from src.dataloader.dataloader import get_MovieLensDataloaders
from src.model.embedding import Simulator, InitialRecommender
from src.trainer.rl_trainer import rl_trainer
from src.trainer.embedding_trainer import Embedding_Trainer

from configs.config import CFG_DICT
from src.utils.file_handler import File_Handler

from src.agent.actor_critic import ActorCriticAgent

import numpy as np
import torch
import pandas as pd

file_handler = File_Handler(log_dir='train_usr/random_split')
logger = file_handler.make_logger_file('src')

In [4]:
load_rawML1M = LoadMovielens1M(path=CFG_DICT['DATASET']['PATH'])
raw_datas = load_rawML1M.load_data()

preprocessML1M = PreprocessMovielens1M(raw_datas)
preprocess_datas = preprocessML1M.apply_prepocess()

ratings, users, movies, histories = preprocess_datas

In [5]:

preprocess_datas = ratings, users, movies, histories

device = 'cuda'

user_simulator = Simulator(emb_size=CFG_DICT['USER_SIMULATOR']['DIM_EMB']).to(device)

user_simlator_datas = get_MovieLensDataloaders(
        model_type = 'USER_SIMULATOR',
        processed_data = preprocess_datas,
        data_split = CFG_DICT['USER_SIMULATOR']['DATA_SPLIT']
    )

user_simulator_trainer = Embedding_Trainer(
    model_type = 'USER_SIMULATOR',
    model = user_simulator,
    loaders = user_simlator_datas,
    device = device
    )

user_simulator = user_simulator_trainer.train()
file_handler.save(user_simulator.state_dict(), 'user_simulator.pkl')

INFO:src.model.embedding:Embedding size : 5
INFO:src.model.embedding:Model Param : 19717
INFO:src.trainer.embedding_trainer:Optimizer : adagrad
INFO:src.trainer.embedding_trainer:Loss : cc
INFO:src.trainer.embedding_trainer:USER_SIMULATOR training, epoch 0
INFO:src.trainer.embedding_trainer:train / loss : 8.595275008998697e-05, auc : 0.5606811641074653
INFO:src.trainer.embedding_trainer:val / loss : 0.00016695788595825434, auc : 0.5752056170513745
INFO:src.trainer.embedding_trainer:test / loss : 0.00016581444477196784, auc : 0.5770837690712732
INFO:src.trainer.embedding_trainer:USER_SIMULATOR training, epoch 1
INFO:src.trainer.embedding_trainer:train / loss : 7.992766827834314e-05, auc : 0.6275331129008639
INFO:src.trainer.embedding_trainer:val / loss : 0.0001541656907647848, auc : 0.6605175672334898
INFO:src.trainer.embedding_trainer:test / loss : 0.00015274231962393968, auc : 0.6656939067665534
INFO:src.trainer.embedding_trainer:USER_SIMULATOR training, epoch 2
INFO:src.trainer.embed